データをカテゴライズしてそれぞれのグループに関数を適用することは、集約や変換と呼ばれ、データ分析のワークフローの中で最も重要な部分になる  
データを読み込み、マージし、データセットを準備した後に行う類似のタスクには、グループごとの統計を行うことや、レポーティングや可視化のためにピボットテーブルを使うこともある  
pandasは柔軟なgroupbyインタフェースを持っており、これを利用することでデータセットを自然な形でスライスし、ダイシングし集計することができる

リレーショナルデータベースとSQLが一般的なのは、データの連結やフィルタリング、変換、集約が容易だからである  
しかし、SQLのようなクエリ言語はグループ演算で実現できることにある程度の規制がある  
一方Pythonとpandasの表現力を使えば、より複雑なグループ演算を行うことができる  
この章では以下のようなことを実現する方法を説明する  
- pandasのオブジェクトを1つあるいは複数のキー(関数や配列、データフレームの列名の形式で指定)を使って分割する方法  
- グループの集約統計量の計算方法。具体的には、個数のカウント、平均値、標準偏差や、ユーザが定義した関数などの計算方法  
- グループ内の変換やその他のデータ操作(正規化、線形回帰、順位、部分集合の選択)
- ピボットテーブルとクロス集計の計算
- 分位点分析やその他のデータから生成されたグループに関する分析

# 10.1  GroupByの仕組み

グループ演算のプロセスを説明するために作られた言葉が分離ー適用ー結合(split-apply-combine)になる  
プロセスの最初の段階では、pandasのオブジェクト(シリーズやデータフレームなど)に格納されているデータが、指定した1つ以上のキーによって分離される  
分離はオブジェクトの特定の軸を使って行われる  
分離が終わったら、それぞれのグループに関数が適用され、新しい値が生成される  
最後に、これらの関数を適用した結果が結合されて結果を戻すオブジェクトに格納される  
戻るオブジェクトの形式は、大抵はデータに行った処理の内容に依存する

グループ化に使用するキーはさまざまな形式を取ることができ、また、すべて同じ型である必要もない  
- 値のリストや配列(グループされる軸と同じ長さのもの)
- データフレームの列名を示す値
- ディクショナリ形式あるいはシリーズ形式(グループされる軸の値とグループ名の間の対応を取れるもの)
- 軸のインデックス、あるいは、各インデックスのラベルに対して呼び出される関数  

この手法のうち後者は3つは、オブジェクトを分離するために使う値の配列を簡単に生成する手段になる  

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({"key1": ["a", "a", "b", "b", "a"],
                   "key2": ["one", "two", "one", "two", "one"],
                   "data1": np.random.randn(5),
                   "data2": np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-1.432148,-0.008802
1,a,two,-0.325900,-0.243402
2,b,one,1.035055,0.473427
3,b,two,-0.186599,-0.466310
4,a,one,0.547272,-0.085318


In [3]:
grouped = df["data1"].groupby(df["key1"])
grouped

key1のラベルでグループ化して、data1列の平均値を計算するには様々な方法があるが、その1つがdata1にアクセスして、groupbyメソッドにグループする列としてkey1の列を指定して呼び出す方法がある  
グループ化された変数は、この時点ではGroupByオブジェクトになっている  
このオブジェクトには実際に計算した結果は含まれておらず、グループキー(df["key"])に関する中間データが含まれている状態になる

In [4]:
grouped.mean()

key1
a   -0.403592
b    0.424228
Name: data1, dtype: float64

GroupByオブジェクトを使って計算を行うには、インスタンスメソッドを呼び出すことでできる  
ここで重要なことは、結果のデータはグループキーに従って集約されており、key1列に含まれる独立した値でインデックスされている新しいシリーズが生成されているところにある  
結果に含まれるインデックスの名前は"key1"になっているが、これはデータフレームの列df["key1"]に対して操作を行ったためである

In [5]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means

key1  key2
a     one    -0.442438
      two    -0.325900
b     one     1.035055
      two    -0.186599
Name: data1, dtype: float64

グループキーとして複数の配列をリストとして渡すと、データを2つのキーを使ってグループ化し、その計算結果のシリーズは階層型インデックスを持ったものになる

In [6]:
means.unstack()

key2,one,two
key1,,
a,-0.442438,-0.325900
b,1.035055,-0.186599


計算結果の階層型インデックスを持ったシリーズに対してunstackメソッドを使うことで階層を下げデータフレームに変換することができる

In [7]:
states = np.array(["Ohio", "California", "California", "Ohio", "Ohio"])
years = np.array([2005, 2005, 2006, 2005, 2006])

In [8]:
df["data1"].groupby([states, years]).mean()

California  2005   -0.325900
            2006    1.035055
Ohio        2005   -0.809373
            2006    0.547272
Name: data1, dtype: float64

グループキーには共通のデータフレームの別の列(シリーズ型)だけでなく、直接関係のない同じ長さ(列の要素数)を持ったシリーズをグループキーに使うこともできる

In [9]:
df.groupby("key1").mean()

,data1,data2
key1,,
a,-0.403592,-0.112507
b,0.424228,0.003559


ほとんどの場合、グループ化に使いたい情報は、データフレームの中のデータとして存在する  
そういう場合には、グループキーとしてそのデータが含まれる列名を渡すことができる  
渡す列名のデータ型が文字列、数値、その他のPythonオブジェクトであっても可能である  
上の結果にはkey2列が含まれていないが、これはdf["key2"]のデータ型が数値ではない列(邪魔な列といわれる)であるために結果から除かれている  
デフォルトではすべての数値列が集約されるが、あとで紹介する方法で列をフィルタリングすることもできる  

In [10]:
df.groupby(["key1", "key2"]).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

GroupByメソッドにはsizeメソッドがあり、各グループのサイズ情報を持つシリーズを返す  
グループキーに含まれる欠損値は結果から除外されることに注意

## 10.1.1  グループをまたいだ繰り返し

グループオブジェクトは繰り返しをサポートし、繰り返しの中ではグループの名前(name)と、その名前に対応するデータ(group)の2つを含むタプルで構成されるシーケンスを生成する

In [11]:
for name, group in df.groupby("key1"):
    print("name:\n", name)
    print("group:\n", group)

name:
 a
group:
   key1 key2     data1     data2
0    a  one -1.432148 -0.008802
1    a  two -0.325900 -0.243402
4    a  one  0.547272 -0.085318
name:
 b
group:
   key1 key2     data1     data2
2    b  one  1.035055  0.473427
3    b  two -0.186599 -0.466310


for文を使ってそれぞれを表示してみると1つ目の変数にグループの名前、2つ目にそのグループに対応するデータがそれぞれに入っていることがわかる

In [12]:
for name, group in df.groupby(["key1", "key2"]):
    print("name:\n", name)
    print("group:\n", group)

name:
 ('a', 'one')
group:
   key1 key2     data1     data2
0    a  one -1.432148 -0.008802
4    a  one  0.547272 -0.085318
name:
 ('a', 'two')
group:
   key1 key2   data1     data2
1    a  two -0.3259 -0.243402
name:
 ('b', 'one')
group:
   key1 key2     data1     data2
2    b  one  1.035055  0.473427
name:
 ('b', 'two')
group:
   key1 key2     data1    data2
3    b  two -0.186599 -0.46631


複数のキーを扱った場合では、グループの名前にあたる部分の変数がキーの値のタプルになって返ってくる

In [13]:
pieces = dict(list(df.groupby("key1")))
pieces["b"]

,key1,key2,data1,data2
2,b,one,1.035055,0.473427
3,b,two,-0.186599,-0.466310


取り出したいデータはどのようなものであっても選択することができる  
便利なものとして、上のようにGroupByオブジェクトをlist関数でリストに変換した後に、dict関数で辞書型に変換することができる  
list型のときはグループの名前とそのグループのpandasオブジェクト(シリーズかデータフレーム)の2つがタプルとして入ったリストで、  
dict型のときはグループの名前がキーで、そのグループのpandasオブジェクトが値になっている辞書になる  
これを利用することで辞書のキーを選択する方法でpandasオブジェクトにアクセスできるようになる

In [14]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [15]:
grouped = df.groupby(df.dtypes, axis=1)

In [16]:
for dtype, group in grouped:
    print("dtype:\n", dtype)
    print("group:\n", group)

dtype:
 float64
group:
       data1     data2
0 -1.432148 -0.008802
1 -0.325900 -0.243402
2  1.035055  0.473427
3 -0.186599 -0.466310
4  0.547272 -0.085318
dtype:
 object
group:
   key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


デフォルトのgroupbyメソッドはaxis=0に設定されているが、axis=1を渡すことで別の方向にグループ化することができる  
上ではデータフレームのdtypeで分割してグループ化している

## 10.1.2  列や列の集合の選択

In [17]:
df.groupby("key1")["data1"]
df.groupby("key1")[["data2"]]

In [18]:
df["data1"].groupby(df["key1"])
df[["data2"]].groupby(df["key1"])

1つの列や複数の列を持つデータフレームから作成されたGroupByオブジェクトに対して、インデックス参照するということは、集約する列を選択するのと同じ効果がある  
つまり上の2つのセルの内容は全く同じもの(シンタックスシュガー)ということになる

In [19]:
df.groupby(["key1", "key2"])[["data2"]].mean() # データフレームになる

data2
key1 key2          
a    one  -0.047060
     two  -0.243402
b    one   0.473427
     two  -0.466310

特に大きなデータセットの場合、わずかな列だけで集約する方が望ましい場合がある  
例えば、data2列だけの平均値を計算して結果をデータフレームで得たければ上のように書くことができる

In [20]:
s_grouped = df.groupby(["key1", "key2"])["data2"] # シリーズになる
s_grouped.mean()

key1  key2
a     one    -0.047060
      two    -0.243402
b     one     0.473427
      two    -0.466310
Name: data2, dtype: float64

このインデックス参照で返されるオブジェクトは、リストや配列を与えたときはグループ化されたデータフレームになり、スカラーとして単独の列名を与えた場合はグループ化されたシリーズになる

## 10.1.3  ディクショナリやシリーズのグループ化

グループ化の情報は、配列以外の形式の場合もある

In [21]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=["a" ,"b", "c", "d", "e"],
                      index=["Joe", "Steve", "Wes", "Jim", "Travis"])
people.iloc[2, [1, 2]] = np.nan # 欠損値を追加
people

,a,b,c,d,e
Joe,0.163349,-0.508681,3.093929,-0.449758,-1.944348
Steve,1.327084,-1.260464,-0.759177,0.263514,1.175510
Wes,-0.531572,NaN,NaN,0.239261,0.120043
Jim,-1.278878,0.611482,0.361103,1.210100,1.133231
Travis,0.581551,-0.059395,1.123314,0.681800,0.357552


In [22]:
mapping = {"a": "red", "b": "red", "c": "blue", "d": "blue", "e": "red", "f": "orange"}

ここでは、どの列をグループ化したいのかを示したマッピング情報があり、そして、複数の列をまとめたグループごとに、合計を算出したいとする  
上がマッピング情報になる

In [23]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,2.644171,-2.289680
Steve,-0.495663,1.242131
Wes,0.239261,-0.411528
Jim,1.571203,0.465835
Travis,1.805114,0.879707


マッピング情報の入ったディクショナリから配列を作ってgroupbyメソッドに渡すこともできるが、簡単に直接ディクショナリをgroupbyに渡すこともできる  
この場合、マッピングに使うディクショナリのキーに使用しないキーが存在しても無視してくれるので大丈夫(上ではfがキーの部分)

In [24]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [25]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


シリーズも固定サイズのマッピングとみなせるため、groupbyに渡して使うこともできる

## 10.1.4  関数を使ったグループ化

ディクショナリやシリーズを使ったグループ化と比べて、Pythonの関数を使う方法は、より汎用的になる  
グループキーとして渡される関数は、インデックスの値ごとに呼び出され、戻り値がグループの名として用いられる  

In [26]:
people.groupby(len).sum()

,a,b,c,d,e
3,-1.647101,0.102802,3.455031,0.999604,-0.691073
5,1.327084,-1.260464,-0.759177,0.263514,1.175510
6,0.581551,-0.059395,1.123314,0.681800,0.357552


上の場合はgroupbyメソッドに関数のlenを渡して、名前の文字列数によってグループ化している

In [27]:
key_list = ["one", "one", "one", "two", "two"]

In [28]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.531572 -0.508681  3.093929 -0.449758 -1.944348
  two -1.278878  0.611482  0.361103  1.210100  1.133231
5 one  1.327084 -1.260464 -0.759177  0.263514  1.175510
6 two  0.581551 -0.059395  1.123314  0.681800  0.357552

配列、ディクショナリ、シリーズでのグループ化と、関数によるグループ化が混在していても、すべてが内部的に配列に変換されるため問題ない

## 10.1.5  インデックス階層によるグループ化

In [29]:
columns = pd.MultiIndex.from_arrays([["US", "US", "US", "JP", "JP"],
                                     [1, 3, 5, 1, 3]],
                                    names=["cty", "tenor"])
columns

MultiIndex([('US', 1),
            ('US', 3),
            ('US', 5),
            ('JP', 1),
            ('JP', 3)],
           names=['cty', 'tenor'])

In [30]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.500348  0.361223  1.476428  0.649780 -0.610025
1      0.127649 -0.940553  2.556079  0.370376  0.062701
2     -0.271402 -0.213320  1.345209 -0.039205 -0.215424
3     -1.521021 -1.665556 -0.363593  0.021791  0.431817

In [31]:
hier_df.groupby(level="cty", axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


階層を持つインデックスを使う際に決定的に便利なのは、軸のインデックスの階層を使って集約ができる点にある  
データフレームを階層ごとに集約するには、groupbyメソッドのキーワード引数lavelに階層を示す番号や階層インデックスの名前を渡すことでできる

In [32]:
hier_df.groupby(level=-1, axis=1).count()

tenor,1,3,5
0,2,2,1
1,2,2,1
2,2,2,1
3,2,2,1


階層を示す番号は外側から0, 1, ...,と振られ、内側からの番号で指定したいときは-1, -2, ...,と指定することでできる

# 10.2  データの集約

集約とは、何らかのデータ変形を行って配列からスカラー値を生成することを指す  
平均、個数のカウント、最小値、最大値などで集約を行う  
最適化済みのGroupByメソッド  

|関数名|説明|
|:-|:-|
|count|グループ内の欠損値以外の値の数|
|sum|欠損値以外の合計|
|mean|欠損値以外の平均|
|median|欠損値以外の算術中央値|
|std, var|バイアスのかかっていない(n-1を分母とした)標準偏差と分散|
|min, max|欠損値以外の最小値と最大値|
|prod|欠損値以外の積|
|frist, last|欠損値以外の最初と最後の値|

上記のような多くの一般的な集約については、すでに最適化された実装が存在するが、これらのメソッドに限定されたものではない  
自分自身で考えた集約処理を使うこともできるし、グループ化されたオブジェクトに定義されているメソッドを追加で呼ぶこともできる  

In [33]:
df

,key1,key2,data1,data2
0,a,one,-1.432148,-0.008802
1,a,two,-0.325900,-0.243402
2,b,one,1.035055,0.473427
3,b,two,-0.186599,-0.466310
4,a,one,0.547272,-0.085318


In [34]:
grouped = df.groupby("key1")

In [35]:
grouped["data1"].quantile(0.9) # quantileはシリーズとデータフレームのインスタンスメソッドで、列の分位点を計算する

key1
a    0.372638
b    0.912890
Name: data1, dtype: float64

上のようにquantileメソッドはGroupByオブジェクトのインスタンスメソッドではないが、シリーズのメソッドであるため使うことができる  
内部的な処理としては、GroupByがシリーズをうまくスライスし、スライスしたそれぞれのピースに対してquantileを呼び出している  
そして、それらの結果を組み立てて結果のシリーズオブジェクトを戻している

In [36]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [37]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.979420,0.234601
b,1.221653,0.939738


自分自身で定義した集約関数を使うには、配列を集約する関数をaggregateあるいはaggメソッドに渡すことでできる

In [38]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.403592  0.991995 -1.432148 -0.879024 -0.325900  0.110686   
b      2.0  0.424228  0.863839 -0.186599  0.118815  0.424228  0.729642   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.547272   3.0 -0.112507  0.119640 -0.243402 -0.164360 -0.085318   
b     1.035055   2.0  0.003559  0.664495 -0.466310 -0.231376  0.003559   

                          
           75%       max  
key1                      
a    -0.047060 -0.008802  
b     0.238493  0.473427

describeのようなメソッドも同様に使うことができる  
しかし、これらのメソッドは厳密には集約ではない

## 10.2.1  列に複数の関数を適用する

In [39]:
tips = pd.read_csv(r".\download_file\examples\tips.csv")

In [40]:
tips["tip_pct"] = tips["tip"] / tips["total_bill"]
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


GroupByオブジェクトでは列ごとに異なる関数を使って集約することや、複数の関数を同時に使って集約することができる

In [41]:
grouped = tips.groupby(["day", "smoker"])

In [42]:
grouped_pct = grouped["tip_pct"]

In [43]:
grouped_pct.agg("mean") # grouped_pct.mean()と同義

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

GroupByオブジェクトのaggメソッドには関数の名前を文字列として渡すこともできる

In [44]:
grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

GroupByのaggメソッドに、関数の名前の文字列や定義した関数をリストとして渡すと、関数名を列名とした、それぞれの計算結果が入ったデータフレームとして返す  
GroupByが列に付けた名前を納得して受け入れる必要はない  
特にlambda関数を使用した場合に列名は"<lambda\>"になり、この名前から内容を知ることは困難になる  
しかし、以下のようにすることで列名を指定したものに変えることができる

In [45]:
grouped_pct.agg([("foo", "mean"), ("bar", np.std)]) # np.stdの部分は"std"でも同じ

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

列名の名前を自由に変更するには、GroupByのaggメソッドに(付けたい名前, 関数)という形のリストにして渡すことでできる  

データフレームに対しては、さらに別のオプションを使うこともできる  
関数のリストを指定することですべての列に関数を適用したり、列ごとに異なる関数を適用したりすることができる

In [46]:
functions = ["count", "mean", "max"]

In [47]:
result = grouped[["tip_pct", "total_bill"]].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

上の場合は、3つの計算をtip_pct列とtotal_bill列に適用している  
結果として得られるデータフレームは階層的な列を持つ  
これは、グループに指定した各列を別々に集約して、その後、結果をひとまとめにして結合するために、concat関数のkeys引数として列名を指定したものと同じ結果になる

In [48]:
ftuples = [("Durchschnitt", "mean"), ("Abweichung", np.var)]

In [49]:
grouped[["tip_pct", "total_bill"]].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

こちらも同じように、列の名前と関数名のタプルのリストを渡すことで列名を指定することができる

In [50]:
grouped.agg({"tip": np.max, "size": "sum"})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

複数の列に対して、別々の関数を適用したい場合は、列名と適用したい関数名をマッピングしたディクショナリをaggメソッドに渡すことでできる

In [51]:
grouped.agg({"tip_pct": ["min", "max", "mean", "std"],
             "size": "sum"})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

ひとつの列に複数の関数を適用する場合は、ディクショナリの値に関数名のリストを渡すことでできる  
少なくとも1つの列に対して複数の関数を適用した場合、データフレームは階層的な列を持つことになる

## 10.2.2  集約されたデータを行インデックスなしで戻す

今までの例では、集約されたデータは自動的にインデックス付けされていて、場合によっては階層型インデックスを持ち、一意なグループキーを基に構成されたものになっていた  
しかし、この動作を無効化することもできる

In [52]:
tips.groupby(["day", "smoker"], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


インデックス付けを無効化するには、groupbyメソッドのキーワード引数のas_indexにFalseを渡すことでできる  
インデックス付けされた結果に対してreset_indexメソッドを呼ぶことでも同じような形式にできるが、as_index=Falseを使った方が不必要な計算が省かれている

# 10.3  applyメソッド : 一般的な分離ー適用ー結合の方法

最も一般的な目的を持つGroupByのメソッドはapplyで、オブジェクトを操作するためのピースに分離しそれぞれのピースに対して渡された関数を適用し、その後それらのピーズを結合する

In [53]:
def top(df, n=5, column="tip_pct"):
    return df.sort_values(by=column)[-n:]

In [54]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


データフレームの特定の列の値で上位のみを返す関数topを上のように定義したとする  

In [55]:
tips.groupby("smoker").apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

定義した関数topをGroupByオブジェクトのapplyメソッドに渡すことで、グループごとに関数を適用し、その後結合したデータフレームを返してくれる  

In [56]:
tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

applyメソッドに渡す関数に引数を指定したい場合は、関数を渡した後に引数を渡すこともできる

## 10.3.1  グループキーの抑制

In [57]:
tips.groupby("smoker", group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


上の結果の前の例では、結果のデータフレームは階層的なインデックスを持ち、グループキーごとにもともとのオブジェクトのインデックスを持つような形式になっていた  
この使用はgroupbyメソッドにキーワード引数でgroup_keysにFalseを渡すことで無効化することができる

## 10.3.2  分位点とビン分析

pandasはcutやqcutなどのツールを持っていて、cutはデータを同じ長さのビンに分割して入れる動作をし、qcutはサンプルデータの分位点でデータを分割する  
これらの関数をgroupbyと組み合わせることでデータセットのビン分析や分位点分析が行いやすくなる

In [58]:
frame = pd.DataFrame({"data1": np.random.randn(1000),
                      "data2": np.random.randn(1000)})

In [59]:
quartiles = pd.cut(frame.data1, 4) # data1列の値を4つのビンに分ける
quartiles[:10] # 先頭10行が4つのビンのどれに属するかを表示

0    (-0.131, 1.449]
1    (-1.71, -0.131]
2    (-0.131, 1.449]
3    (-3.297, -1.71]
4    (-1.71, -0.131]
5    (-0.131, 1.449]
6    (-0.131, 1.449]
7    (-1.71, -0.131]
8     (1.449, 3.029]
9    (-1.71, -0.131]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.297, -1.71] < (-1.71, -0.131] < (-0.131, 1.449] < (1.449, 3.029]]

上では、シンプルでランダムな数値を持つデータセットを用意して、cut関数を使って同じ長さのビンに分割している

In [60]:
def get_stats(group):
    return {"min": group.min(), "max": group.max(), "count": group.count(), "mean": group.mean()}

In [61]:
grouped = frame.data2.groupby(quartiles)

In [62]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.297, -1.71]",-2.022886,2.177728,47.0,-0.030311
"(-1.71, -0.131]",-2.758432,3.080435,378.0,0.013950
"(-0.131, 1.449]",-3.257345,3.247680,510.0,0.041418
"(1.449, 3.029]",-2.280249,1.743297,65.0,-0.217417


cut関数で返されるCategoricalオブジェクトは、groupbyメソッドに渡すこともできる

In [63]:
grouping = pd.qcut(frame.data1, 10, labels=False)

In [64]:
grouped = frame.data2.groupby(grouping)

In [65]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.022886,2.572159,100.0,0.028618
1,-2.597418,2.209638,100.0,0.060620
2,-2.758432,3.080435,100.0,-0.076314
3,-2.285777,2.281138,100.0,0.005216
4,-2.134287,2.591326,100.0,-0.002075
5,-1.929690,2.269342,100.0,0.177914
6,-2.407528,2.114637,100.0,-0.054963
7,-2.622210,3.079196,100.0,0.081615
8,-3.257345,3.247680,100.0,-0.024313


cut関数では各ビンが同じ長さであったためデータ数がばらばらだったが、qcut関数を使うことで分位点に基づいて同じ個数のデータが入ったビンにできる  
qcut関数にlabel=Falseを渡して分位点の数値を取得することができる

## 10.3.3  グループ固有の値で欠損値を埋める

欠損値をクリーニングする際にdropnaを使って欠損値を置き換える方法もあるが、固定値やデータから導出した値で欠損値を穴埋めしたいときもある  
fillnaメソッドを使うことでこれを実現することができる

In [66]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -2.532740
2         NaN
3    1.576333
4         NaN
5   -0.400576
dtype: float64

In [67]:
s.fillna(s.mean())

0   -0.452327
1   -2.532740
2   -0.452327
3    1.576333
4   -0.452327
5   -0.400576
dtype: float64

上ではfillnaメソッドにシリーズの平均値を渡すことで欠損値を穴埋めしている

In [68]:
states = {"Ohio", "New York", "Vermont", "Florida", "Oregon", "Nevada", "California", "Idaho"}

In [69]:
group_key = ["East"] * 4 + ["West"] * 4

In [70]:
data = pd.Series(np.random.randn(8), index=states)
data

Oregon        2.287705
Vermont       1.762318
New York      0.766489
Ohio         -1.080925
California    0.420041
Idaho        -1.079274
Nevada        0.898800
Florida       0.642884
dtype: float64

欠損値を埋める値をグループによって変えたい場合を考えてみる  
これを行うためには、データをグループ分けしてそれぞれのグループに対してfillnaを使う関数をapplyメソッドに渡すことになる

In [71]:
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data

Oregon        2.287705
Vermont            NaN
New York      0.766489
Ohio         -1.080925
California    0.420041
Idaho              NaN
Nevada             NaN
Florida       0.642884
dtype: float64

In [72]:
data.groupby(group_key).mean()

East    0.657756
West    0.531463
dtype: float64

まずは、欠損値を埋める値を決めるためにグループの平均値を求めている

In [73]:
fill_mean = lambda g: g.fillna(g.mean())

In [74]:
data.groupby(group_key).apply(fill_mean)

Oregon        2.287705
Vermont       0.657756
New York      0.766489
Ohio         -1.080925
California    0.420041
Idaho         0.531463
Nevada        0.531463
Florida       0.642884
dtype: float64

上のように欠損値を埋めるための関数を定義して、GroupByオブジェクトのapplyメソッドに渡すことでそれぞれのグループごとに平均値を求め欠損値を埋めてくれる

In [75]:
fill_values = {"East": 0.5, "West": -1}

In [76]:
fill_func = lambda g: g.fillna(fill_values[g.name])

In [77]:
data.groupby(group_key).apply(fill_func)

Oregon        2.287705
Vermont       0.500000
New York      0.766489
Ohio         -1.080925
California    0.420041
Idaho        -1.000000
Nevada       -1.000000
Florida       0.642884
dtype: float64

平均値ではなくグループごとに指定した値で埋めたい場合は、fillnaメソッドでグループが内部的に持っているname属性を呼び出し、ディクショナリのキーとして呼び出すことで対応する値で埋めることができる

## 10.3.4  ランダムサンプリングと順列

モンテカルロシミュレーション(乱数を利用して事象を確率的にシミュレーションする方法)をする場合や、その他の応用において、巨大なデータセットからランダムにサンプルデータを抽出したいことがある  
この抽出を行う方法は数多くあるが、ここではシリーズのsampleメソッドを使う

In [78]:
suits = ["H", "S", "C", "D"]
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ["A"] + list(range(2, 11)) + ["J", "Q", "K"]
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)
deck = pd.Series(card_val, index=cards)

In [79]:
deck

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
QH     10
KH     10
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
8S      8
9S      9
10S    10
JS     10
QS     10
KS     10
AC      1
2C      2
3C      3
4C      4
5C      5
6C      6
7C      7
8C      8
9C      9
10C    10
JC     10
QC     10
KC     10
AD      1
2D      2
3D      3
4D      4
5D      5
6D      6
7D      7
8D      8
9D      9
10D    10
JD     10
QD     10
KD     10
dtype: int64

デモンストレーションのためにブラックジャックなどで使われるシリーズを用意した  
ここではわかりやすくするためにエースは1としている

In [80]:
def draw(deck, n=5):
    return deck.sample(n)

In [81]:
draw(deck)

5C      5
6D      6
AD      1
5D      5
10D    10
dtype: int64

シリーズの中からデータをランダムに抽出する関数を作成するには上のようにする  
シリーズオブジェクトのsampleメソッドを呼び、抽出する要素数を渡すことでできる(何も渡さなければ1つだけ抽出される)

In [82]:
get_suit = lambda card: card[-1] # 最後の文字(HSCD)でグループ分けする

In [83]:
deck.groupby(get_suit).apply(draw, n=2)

C  8C      8
   3C      3
D  6D      6
   3D      3
H  8H      8
   10H    10
S  9S      9
   5S      5
dtype: int64

それぞれのスートからランダムな2つを抽出する場合は上のようにすることでできる  
まずスートごとにグループ分けされたGroupByオブジェクトを作成し、applyメソッドで抽出するための関数を適用している

In [84]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

2C     2
6C     6
QD    10
2D     2
8H     8
7H     7
6S     6
7S     7
dtype: int64

上のようにしてグループ名のインデックスを無効化することもできる

## 10.3.5  例:グループの加重平均と相関

groupbyの分離ー適用ー結合パラダイムのもとでは、グループの加重平均値のような、データフレームの複数の列間の操作や2つのシリーズの操作も可能になる

In [85]:
df = pd.DataFrame({"category": ["a", "a", "a", "a", "b", "b", "b", "b"],
                   "data": np.random.randn(8),
                   "weights": np.random.rand(8)})

In [86]:
df

,category,data,weights
0,a,-0.928131,0.211284
1,a,0.090188,0.617979
2,a,-1.164058,0.023587
3,a,0.949140,0.173016
4,b,0.853177,0.525478
5,b,0.192535,0.199481
6,b,0.094837,0.387692
7,b,1.408370,0.021321


グループキー、値、何らかの重みを含む次のようなデータセットを例に見ていく

In [87]:
grouped = df.groupby("category")

In [88]:
get_wavg = lambda g: np.average(g["data"], weights=g["weights"]) # weightsを指定することでそれぞれの要素に対する重みづけをしている

In [89]:
grouped.apply(get_wavg)

category
a   -0.003514
b    0.488132
dtype: float64

データをカテゴリ別のグループで加重平均を求めるには次のようにする  
まずcategory列でグループ分けしたGroupByオブジェクトを作成し、apllyメソッドを使って加重平均を求める関数を渡している

In [90]:
close_px = pd.read_csv(r".\download_file\examples\stock_px_2.csv", parse_dates=True, index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [91]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


別の例として上の金融データを使う  
いくつかの株式とS&P500インデックスの終始データになる  
ここで調べるのは、日時の利益(パーセント変化から算出)とSPXとの年次の相関を調べる

In [92]:
spx_corr = lambda x: x.corrwith(x["SPX"])

まず、特定の列と"SPX"との相関を計算する関数を作っている

In [93]:
rets = close_px.pct_change().dropna() # pct_changeメソッドはデフォルトだと上の列との差分を取る

データフレームのpct_changeメソッドを使って変化を計算して、できた欠損値を削除している

In [94]:
get_year = lambda x: x.year # rets.index.yearでインデックスの年数を返す関数

In [95]:
by_year = rets.groupby(get_year)

In [96]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


変化を計算したデータフレームをグループ化するために、各datetimeラベルのyear属性を返す関数を作成し、groupbyメソッドに渡している  
最後にGroupByオブジェクトのapplyメソッドに"SPX"列との相関を計算するための関数spx_corrを渡して計算している

In [97]:
by_year.apply(lambda g: g["AAPL"].corr(g["MSFT"]))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

ちなみに、列間の相関を計算するには上のように、特定の列のcorrメソッドに特定の列を指定した関数を使うことでできる

## 10.3.6  例:グループ指向の線形回帰

関数の戻り値がpandasオブジェクトやスカラー値である限りは、groupbyはより複雑なグループ指向の統計分析に使うことができる  
たとえば以下のようにregress関数を定義することで最小二乗法(OLS)によるデータの線形回帰を求めることができる

In [98]:
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X["intercept"] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [99]:
by_year.apply(regress, "AAPL", ["SPX"]) # regress(by_year, "AAPL", ["SPX"])という形になる

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


上ではAAPLとSPXの年毎の線形回帰の結果を求めている

# 10.4  ピボットテーブルとクロス集計

ピボットテーブルは、表計算プログラムやその他のデータ分析ソフトウェアでよく見られるデータの要約ツールになる  
ピボットテーブルでは、データは1つ以上のキーによってテーブル形式のデータを要約し、よく特定のグループキーを行に対して、また別のグループキーを列に対して整理し、全体として長方形の形にデータを整形する  
pandasにおけるピボットテーブルは、groupbyの機能と階層型のインデックスを操作する再形成機能の組み合わせで実現されている  
データフレームには、pivot_tableメソッドやpandas.pivot_table関数があり、groupby関数に便利なインタフェースを提供するのに加えて、小計を計算することもできる  
これはpivot_table関数のmarginオプションとして知られている  

In [100]:
tips.tail()

,total_bill,tip,smoker,day,time,size,tip_pct
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204
243,18.78,3.00,No,Thur,Dinner,2,0.159744


In [101]:
tips.pivot_table(index=["day", "smoker"]) # pivot_tableはデフォルトでは平均値を計算する

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

上では"day"と"smoker"によって整理されたグループの平均値を計算している  
pivot_tableメソッドで行のグループ化に使う列の指定はキーワード引数のindexに列名を渡すことででき、複数指定する場合はリストにまとめて渡す  
これはgroupbyメソッドを使っても同じようにできる

In [102]:
tips.pivot_table(["tip_pct", "size"], index=["time", "day"], columns="smoker")

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

上では"tip_pct"と"size"列を計算対象とし、グループ化に使う行を"time"と"day"、グループ化に使う列に"smoker"をそれぞれ指定している  
列のグループ化に使う列の指定はキーワード引数のcolumnsに列名を渡すことでできる

In [103]:
tips.pivot_table(["tip_pct", "size"], index=["time", "day"], columns="smoker", margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

pivot_tableメソッドのキーワード引数marginsにTrueを渡すことで、Allという列と行が追加され、その行や列にあるデータの小計(集計値)が入る  
上の場合All列では、smokerの値がYesかNoを区別なく計算した平均値が入っている  
All行は、行が持つ2つの階層のすべてを同じグループとした場合の平均値を求めている

In [104]:
tips.pivot_table("tip_pct", index=["time", "smoker"], columns="day", aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

デフォルトの平均値を求める以外の関数を適用するには、キーワード引数のaggfuncに関数を渡すことでできる  
aggfuncにlen関数を渡すことでグループのサイズをクロス集計(個数や頻度の集計)した結果を得ることができる

In [105]:
tips.pivot_table("tip_pct", index=["time", "size", "smoker"], columns="day", aggfunc="mean", fill_value=0) # "mean"はnp.meanと同義

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

いくつかの組み合わせが空白、もしくは欠損値だった場合、キーワード引数のfill_valueに埋めたい文字列や数値を渡すことで埋めることができる

pivot_tableのオプション一覧  

|オプション|説明|
|:-|:-|
|values|集約する(1つかそれ以上の)列の名称。デフォルトでは、すべての数値列を集約する|
|index|ピボットテーブルの行でグループ化するための(データフレームの)列名か、その他のグループキー|
|coumns|ピボットテーブルの列でグループ化するための列名か、その他のグループキー|
|aggfunc|集約に用いる関数や、関数のリスト。"mean"がデフォルト。groupbyの文脈で正しい関数はどの関数でも使える|
|fill_value|結果のテーブルで、欠損値を置き換えるための値|
|dropna|Trueを指定すると、すべての要素がNAである列を除外する|
|margins|行・列の小計や合計を追加する。デフォルトはFalseで追加しない|

## 10.4.1  クロス集計: crosstabメソッド

クロス集計はピボットテーブルの特殊なケースで、グループの出現頻度を計算するものになる

In [106]:
data = pd.DataFrame([range(1, 11),
                     ["USA", "Japan", "USA", "Japan", "Japan", "Japan", "USA", "USA", "Japan", "USA"],
                     ["Right-handed", "Left-handed", "Right-handed", "Right-handed", "Left-handed",
                     "Right-handed", "Right-handed", "Left-handed", "Right-handed", "Right-handed"]],
                    index=["Samples", "Nationality", "Handedness"]).T

In [107]:
data

,Samples,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


上のデータをpivot_tableを使って集計することもできるが、pandasのcrosstab関数を使うことでもできる

In [108]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


pandasのcrosstab関数に、集計の行に使いたい列と、集計の列に使いたい列を渡すことでクロス集計を行うことができる  
こちらにもキーワード引数のmerginsがあり、小計を出すこともできる

In [109]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

crosstab関数の最初の2つの引数は、配列、シリーズ、配列のリストを使うことができ、配列のリストを渡して複数をキーを選択すると階層型インデックスのデータフレームになる